# Pipeline for estimating lakes area

### 0. Import libraries and load data

In [22]:
%reload_ext autoreload
%matplotlib inline
%autoreload 2

from src.model_trainer import *
from src.data_loader import *
from src.lake_analyzer import *

import numpy as np
import pandas as pd
from tqdm import tqdm

TRAIN_LAKES = ["george", "mono", "melincue"]
TEST_LAKES = ["walker"]
THRESHOLDS_NUMBER = 10


In [2]:
train_data, data_indices = load_lakes(TRAIN_LAKES)
test_data, _ = load_lakes(TEST_LAKES)

### 1. Train and optimize lake detector with cross-validation

In [3]:
f1_scores = []
kappa_scores = []

# define the space for thresholds
thresholds = [
    np.percentile(get_water_index_values(train_data)[2], percentile)
    for percentile in np.linspace(20, 100, THRESHOLDS_NUMBER)
]

# go over each threshold
for threshold in tqdm(thresholds):
    # compute the average metric for the current threshold
    avg_f1, avg_kappa = compute_cross_validation_scores(train_data, data_indices, threshold)

    f1_scores.append(avg_f1)
    kappa_scores.append(avg_kappa)

# display a data frame that contains the results
threshold_scores = pd.DataFrame(
    {"threshold": thresholds, "f1_score": f1_scores, "kappa_score": kappa_scores}
).sort_values("f1_score", ascending=False)
threshold_scores



KeyboardInterrupt: 

In [4]:
# get predictions for the train dataset
best_threshold = 0.129791 # threshold_scores["threshold"].iloc[0]
train_detections = predict(train_data, best_threshold)

### 2. Test lake detector

In [5]:
# get predictions for the test dataset
test_detections = predict(test_data, best_threshold)

# compute the test metrics with the current threshold
avg_f1, avg_kappa = compute_cross_validation_scores(
    test_data, [list(range(len(test_data)))], best_threshold, test=True
)
print(f"Test F1: {avg_f1}\nTest kappa: {avg_kappa}")


Test F1: 0.6826622223650408
Test kappa: 0.3428896767498583


### 3. Analyze lake evolution

In [23]:
analyze_lake("george", train_data, train_detections)


100%|██████████| 116/116 [00:09<00:00, 12.68it/s]


interactive(children=(IntSlider(value=0, description='index', max=115), Output()), _dom_classes=('widget-inter…

interactive(children=(IntSlider(value=0, description='index', max=115), Output()), _dom_classes=('widget-inter…